##total count of bikes rented during each hour

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

# loading dataset

In [ ]:
path="../bikes/train_bikes.csv"
dataset=pd.read_csv(path)
df=dataset.copy()
df.head()

# Basic statistic 

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

# Numerical features

In [ ]:
numerical_features=[feature for feature in df.columns if df[feature].dtype!='O']

In [ ]:
df[numerical_features]

In [ ]:
df['season'].unique()<25

## Discreate Features

In [ ]:
discreate_feature=[feature for feature in numerical_features if len(df[feature].unique())<25]

In [ ]:
for feature in discreate_feature:
    data=df.copy()
    data.groupby(feature)['count'].median().plot.bar()
    plt.title(feature)
    plt.show()

## Contious feature

In [ ]:
continous_feature=[feature for feature in numerical_features if len(df[feature].unique())>=25]

In [ ]:
df[continous_feature]

In [ ]:
for feature in continous_feature:
    data=df.copy()
    plt.scatter(data[feature],data['count'])
    plt.title(feature)
    plt.xlabel(feature)
    plt.ylabel('count')
    plt.show()

In [ ]:
# hist plot
for feature in continous_feature:
    data=df.copy()
    data[feature].hist(bins=25)
    plt.title(feature)
    plt.show()

# Logarithmic transformation

In [ ]:
for feature in continous_feature:
    data=df.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]=np.log(data[feature])
        data['count']= np.log(data['count'])
        plt.scatter(df[feature],df['count'])
        plt.title(feature)
        plt.show()

# Outliers

In [ ]:
for feature in numerical_features:
    data=df.copy()
    data.boxplot(feature)
    plt.title(feature)
    plt.show()

# Missing value

In [ ]:
df.isnull().sum()

In [ ]:
features_with_na=[feature for feature in df.columns if df[feature].isnull().mean()]
for feature in features_with_na:
    print("missing value of {} is {}".format(feature,np.round(df[feature].isnull().mean())*100,4),'% missing')
else:
    print('there is no missing value')

In [ ]:
df.head()

# Converting datetime to Year, Month, Day, Hours

In [ ]:
df['datetime_new']=pd.to_datetime(df['datetime'])

In [ ]:
df.head()

In [ ]:
df['Year']=df['datetime_new'].map(lambda x: x.year)
df['Month']=df['datetime_new'].map(lambda x:x.month)
df['Day']=df['datetime_new'].map(lambda x: x.day)
df['Hour']=df['datetime_new'].map(lambda x:x.hour)

In [ ]:
df.sample(4)

In [ ]:
df['Day'].unique()

## Define hourly plot for working day and non working day in whole year 
 here, 1:working day
       0: non working day

In [ ]:
df.groupby(['Hour','workingday'])['count'].sum().unstack().plot.bar(figsize=(15,4), width=0.9)

In [ ]:
#df[df.Year==2011]

## Define hourly plot for working day and non working day in 2011 and 2012

In [ ]:
def define_hourly_work_plot(df, year=None,agg='sum'):
    data=df[df.Year==year]
    hourly_data=data.groupby(['Hour','workingday'])['count'].agg(agg).unstack()
    hourly_plot=hourly_data.plot(kind='bar', ylim=(0,110000),
                                 figsize=(15,5),
                                 width=0.9,
                                 title='Year={}'.format(year))

    return hourly_plot
    

In [ ]:
define_hourly_work_plot(df,year=2011)
define_hourly_work_plot(df,year=2012)

# Define hourly plot in 2011 and 2012

In [ ]:
df.groupby(['Hour'])['count'].sum().plot.bar()

In [ ]:
def define_hourly_plot(df, year=None,agg='sum'):
    data=df[df.Year==year]
    hourly_data=data.groupby(['Hour'])['count'].agg(agg)
    hourly_plot=hourly_data.plot(kind='bar', ylim=(0,140000),
                                 figsize=(15,5),
                                 width=0.9,
                                 title='Year={}'.format(year))

    return hourly_plot

In [ ]:
define_hourly_plot(df,year=2011)

In [ ]:
define_hourly_plot(df,year=2012)

# Comparison Year to Year data

In [ ]:
def houly_plot_years(attr,title):
    data=df.copy()
    hourly_data_year=data.groupby([attr,'Year'])['count'].agg('sum').unstack()
    hourly_data_year_plot=hourly_data_year.plot(kind='bar',figsize=(15,5),width=0.9, title=title)
    
    return hourly_data_year_plot

In [ ]:
df['Hour'].unique()

In [ ]:
houly_plot_years('Hour',"Rent bikes per hour in 2011 and 2012")
houly_plot_years('Day',"Rent bikes per Day in 2011 and 2012")
houly_plot_years('Month',"Rent bikes per Month in 2011 and 2012")

# Daywise count for specific hour

In [ ]:
df[(df['Year']==2011) & (df['Month']==1)& (df['Day']==1)&(df['Hour']==0)]['count'].values

In [ ]:
data=df.copy()
data1=data[(data['Year']==2011) & (data['Month']==1)]
data1[(data1.Day==1)&(data1.Hour==0)]['count'].values

In [ ]:
data1[(data1.Day==2)&(data1.Hour==0)]['count'].values

In [ ]:
data1[(data1.Day==19)&(data1.Hour==0)]['count'].values

In [ ]:
data1[(data1.Day==20)&(data1.Hour==0)]['count'].values

In [ ]:
days={}
data=df.copy()
data1=data[(data['Year']==2011) & (data['Month']==1)]

for day in range(1,20):
    days[day]=data1[(data1.Day==day)&(data1.Hour==0)]['count'].values
#print(hours)

plt.figure(figsize=(15,5))
for key,val in days.items():
    #print(key,val)
    plt.bar(key, val)
    plt.xticks(range(0,20))
    plt.xlabel('Day')
    plt.ylabel('count')
    plt.title('bikes rent at 1st hour of January, 2011')

In [ ]:
days={}
data=df.copy()
data1=data[(data['Year']==2011) & (data['Month']==1)]
for i in range(1,24):
    for day in range(1,20):
        days[day]=data1[(data1.Day==day)&(data1.Hour==i)]['count'].values
    #print(hours)

    plt.figure(figsize=(15,5))
    for key,val in days.items():
        #print(key,val)
        plt.bar(key, val)
        plt.xticks(range(0,20))
        plt.xlabel('Day')
        plt.ylabel('count')
        plt.title('bikes rent at {} hour of January, 2011'.format(i))

# Using Box Plot

# Yearwise hourly count

In [ ]:
def box_year_hourly(df,message=''):
    data=df.copy()
    hours={}
    for hour in range(24):
        hours[hour]=data[data.Hour==hour]['count'].values
    
    plt.figure(figsize=(20,10))
    plt.xlabel('Hours')
    plt.ylabel('Count rent')
    plt.title('count vs hours\n'+message)
    plt.boxplot([hours[hour] for hour in range(24)])
    
    axis=plt.gca()
    axis.set_ylim([1,1100])

In [ ]:
# box plot for hourly count for the mentioned year
box_year_hourly(df[df['Year']==2011],'year 2011')
box_year_hourly(df[df['Year']==2012],'year 2012')

In [ ]:

box_year_hourly( df[df.workingday == 1], 'working day') # plotting hourly count of rented bikes for working days for a given year
box_year_hourly( df[df.workingday == 0], 'non working day') # plotting hourly count of rented bikes for non-working days for a given year

In [ ]:
# an Hour bs Count Graph depicting average bike demand based on the hour 
figure,axes = plt.subplots(figsize = (10, 5))
hours = df.groupby(["Hour"]).agg("mean")["count"]  
hours.plot(kind="line", ax=axes) 
plt.title('Hours VS Counts')
axes.set_xlabel('Time in Hours')
axes.set_ylabel('Average of the Bike Demand')
plt.show()

# define range of hour values

In [ ]:
def range_to_group(x):
    if 0<=x<6:
        return 0
    elif 6<=x<13:
        return 1
    elif 13<=x<19:
        return 2
    elif 19<=x<24:
        return 3

In [ ]:
df['Hour2']=df['Hour'].apply(range_to_group)

In [ ]:
# an Hour bs Count Graph depicting average bike demand based on the hour 
figure,axes = plt.subplots(figsize = (10, 5))
hours = df.groupby(["Hour2"]).agg("mean")["count"]  
hours.plot(kind="line", ax=axes) 
plt.title('Hours VS Counts')
axes.set_xlabel('Time in Hours')
axes.set_ylabel('Average of the Bike Demand')
plt.show()

In [ ]:
df.sample(5)

# What is the relationship between temp and count

In [ ]:
df['temp'].hist()

In [ ]:
df.groupby(['temp'])['count'].mean().plot()
plt.show()

In [ ]:
df['atemp'].hist()

In [ ]:
df.groupby(['atemp'])['count'].mean().plot()
plt.show()

In [ ]:
df.groupby(['humidity'])['count'].mean().plot()
plt.ylabel('count')
plt.show()

In [ ]:
df.groupby(['windspeed'])['count'].mean().plot()
plt.ylabel('count')
plt.show()

In [ ]:
df.groupby(['casual'])['count'].mean().plot()
plt.ylabel('count')
plt.show()

In [ ]:
df.groupby(['registered'])['count'].mean().plot()
plt.ylabel('count')
plt.show()

In [ ]:
season	holiday	workingday	weather

In [ ]:
df.groupby(['season'])['count'].mean().plot()
plt.ylabel('count')
plt.show()

In [ ]:
df.groupby(['holiday'])['count'].mean().plot()
plt.ylabel('count')
plt.show()

In [ ]:
df.groupby(['workingday'])['count'].mean().plot()
plt.ylabel('count')
plt.show()

In [ ]:
df.groupby(['weather'])['count'].mean().plot()
plt.ylabel('count')
plt.show()

In [ ]:
df['holiday'].unique()

In [ ]:
sns.pairplot(df)